In [ ]:
import numpy as np

In [ ]:
#FEA Code
def tri3_elem_arrays(nen, xe, ye, k, Q):
    x1,x2,x3 = xe
    y1,y2,y3 = ye
    A = x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2)
    A *= 1/2
    F = np.array([1,1,1], dtype = 'float')
    for i in range (len(F)):
      F[i] *= (1/3) * Q * A
    fe = F

    dN1_dx = (y2-y3)
    dN1_dy = x3-x2
    dN2_dx = y3-y1
    dN2_dy = x1-x3
    dN3_dx = y1-y2
    dN3_dy = (x2-x1)
    coef = (1/(4*A))*k

    K11 = (dN1_dx * dN1_dx) + (dN1_dy * dN1_dy)
    K12 = (dN1_dx * dN2_dx) + (dN1_dy * dN2_dy)
    K13 = (dN1_dx * dN3_dx) + (dN1_dy * dN3_dy)
    K21 = K12
    K22 = (dN2_dx * dN2_dx) + (dN2_dy * dN2_dy)
    K23 = (dN2_dx * dN3_dx) + (dN2_dy * dN3_dy)
    K31 = K13
    K32 = K23
    K33 = (dN3_dx * dN3_dx) + (dN3_dy * dN3_dy)
    ke = np.array([[K11,K12,K13], [K21, K22, K23], [K31, K32, K33]], dtype = 'float')
    ke *= coef
    return ke, fe

In [ ]:
ke, fe = tri3_elem_arrays(1, [0,1,0], [0,1,1], 1, 1)

In [ ]:
print(fe)
print(ke)

[0.16666667 0.16666667 0.16666667]
[[ 0.5  0.  -0.5]
 [ 0.   0.5 -0.5]
 [-0.5 -0.5  1. ]]


In [ ]:
def tri3_assemble_global_arrays(numnp, numel, nen, p, LM, k, Q):

    K = np.zeros((numnp, numnp))
    F = np.zeros(numnp)



    #Loop over elements
    for i_elem in range(numel):


        # Get the coordinates of this element using LM
        xe = p[LM[:,i_elem] - 1, 0]
        ye = p[LM[:,i_elem] - 1, 1]
        #print(xe)

        # Compute the element arrays using tri3_elem_arrays function
        ke, fe = tri3_elem_arrays(nen, xe, ye, k, Q)
        print(ke)
        print(fe)

        # Add the contribution of this element to the global arrays
        for i in range(nen):
            for j in range(nen):
                K[LM[i, i_elem] - 1, LM[j, i_elem] - 1] += ke[i, j]
            F[LM[i, i_elem] - 1] += fe[i]
    #print(K)
    #print(F)
    return K,F

In [ ]:
K,F = tri3_assemble_global_arrays(6, 4, 3, np.array([[0,0], [1/2,0], [1/2,1/2], [1,0], [1,1/2], [1,1]]), np.array([[1,2,2,3],  [2,5,4,5], [3,3,5,6]]), 1, 1)
print(K)

[[ 0.5 -0.5  0. ]
 [-0.5  1.  -0.5]
 [ 0.  -0.5  0.5]]
[0.04166667 0.04166667 0.04166667]
[[ 0.5  0.  -0.5]
 [ 0.   0.5 -0.5]
 [-0.5 -0.5  1. ]]
[0.04166667 0.04166667 0.04166667]
[[ 0.5 -0.5  0. ]
 [-0.5  1.  -0.5]
 [ 0.  -0.5  0.5]]
[0.04166667 0.04166667 0.04166667]
[[ 0.5 -0.5  0. ]
 [-0.5  1.  -0.5]
 [ 0.  -0.5  0.5]]
[0.04166667 0.04166667 0.04166667]
[[ 0.5 -0.5  0.   0.   0.   0. ]
 [-0.5  2.  -1.  -0.5  0.   0. ]
 [ 0.  -1.   2.   0.  -1.   0. ]
 [ 0.  -0.5  0.   1.  -0.5  0. ]
 [ 0.   0.  -1.  -0.5  2.  -0.5]
 [ 0.   0.   0.   0.  -0.5  0.5]]


In [ ]:
K[3,:] = 0
K[3,3] = 1
F[3] = 0

K[4,:] = 0
K[4,4] = 1
F[4] = 0

K[5,:] = 0
K[5,5] = 1
F[5] = 0

T = np.linalg.solve(K,F)
print(T)

[0.3125     0.22916667 0.17708333 0.         0.         0.        ]


In [ ]:
def tri3_apply_BCs(nodeFlag, To, Ti, K, F):
    K,F = K,F
    for i in range(len(nodeFlag)):
      if nodeFlag[i] == 0:
        continue
      elif nodeFlag[i] == 1:
        K[i,:] = 0
        K[i][i] = 1
        F[i] = To
      elif nodeFlag[i] == 2:
        continue
      elif nodeFlag[i] == 3:
        K[i,:] = 0
        K[i][i] = 1
        F[i] = Ti


    return K,F

In [ ]:
#Solve 2D PDE
def laser_heat_2d(coord_lim, N_array, t_end, Nt, rho, c, k, P, w, x_start, x_end, vL, T_bar):
    xmin, xmax = coord_lim[0,:]
    ymin, ymax = coord_lim[1,:]
    Nx, Ny = N_array
    x = np.linspace(xmin, xmax, Nx+1)
    y = np.linspace(ymin, ymax, Ny+1)
    t = np.linspace(0, t_end, Nt+1)

    dt = t[1] - t[0]
    dx = x[1] - x[0]
    dy = y[1] - y[0]

    xL = lambda t: x_start + vL*t


    alpha = (k/(rho*c))

    # Initialize
    T = np.zeros((Nx+1, Ny+1, Nt+1))
    T[:, :, 0] = T_bar

    for n in range(0,Nt):
        for i in range(1, Nx):
            for j in range(1, Ny):
                x_comp = T[i+1, j, n] - 2*T[i,j,n] + T[i-1, j, n]
                y_comp = T[i,j+1, n] - 2*T[i, j, n] + T[i, j-1, n]
                T[i , j, n+1] = T[i,j,n] + dt*alpha*((x_comp/(dx**2)) + (y_comp/(dy**2)))

        #Boundary Conditions
        #Neumann
        if 0 <= (t[n]) <= ((x_end - x_start) / vL):
          #xl = xL(n)
          T[1:-1, -1, n + 1] = (T[1:-1, -2, n+1]) + (dy*((P/(w*np.sqrt(2*np.pi))) * np.exp((-(x[1:-1]-xL(t[n]))**2) / (2*(w**2))))) / k
        else:
          T[1:-1 , -1, n + 1] = T[1:-1, -2, n+1]

        T[-1, :, n + 1] = T_bar #right end
        T[0, :, n + 1] = T_bar #left end
        #T[1:-1 , -1, n + 1] = T[1:-1, -2, n+1]
        T[:, 0, n + 1] = T_bar # bottom


    dt_max = 1/(2*alpha*((1/(dx**2)) + (1/(dy**2))))




    return dt_max, x, y, t, T

In [ ]:
#coord_lim =[[0.   0.05]
 #[0.   0.01]],
t_end=2
Nt=4000
rho=7900
c=470
k=48
P=5000
w=0.001
x_start=0.005
x_end=0.045
vL=0.02
T_bar=25
coord_lim = np.array([[0,0.05], [0,0.01]], dtype = 'float')
dt_max, x, y, t, T = laser_heat_2d(coord_lim, [250,50], t_end, Nt, rho, c, k, P, w, x_start, x_end, vL, T_bar)


In [ ]:
print(dt_max)
print(T)

In [ ]:
# ======================= Suggested plotting code ==========================
import matplotlib.pyplot as plt
X, Y = np.meshgrid(x, y)
tplot_idx = 4000 # Time index to plot; must be integer between 0 and Nt inclusive, varied for each time step plotted
plt.figure()
levels = np.linspace(25, np.max(T[:,:,:]), 21)
plt.contourf(X, Y, T[:,:,tplot_idx].T, levels=levels, cmap='rainbow')
cbar = plt.colorbar()
cbar.ax.set_ylabel('Temperature', rotation=90)
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Temperature distribution in the domain at $t=$'+str(t[tplot_idx]))

# # Save
# plt.savefig('contour.png')
# # Display
# plt.show()

In [ ]:
plt.plot(t, T[126][-1], label = 'x = Lx/2, y = Ly')
plt.plot(t, T[126][26], label = 'x = Lx/2, y = Ly/2')
plt.xlabel('t')
plt.ylabel('T')
plt.legend()
plt.grid()
plt.title('Temperature Profile at 2 Points of the Domain')

In [ ]:
print(np.amax(T))

103.86403479603047


In [ ]:
print(np.shape(T))

(251, 51, 4001)


In [ ]:
t_end=2
Nt=2000
N_array = [100,20]
rho=7900
c=470
k=48
P=np.array([1250, 2500, 3750, 5000, 6250, 7500])
w=0.001
x_start=0.005
x_end=0.045
vL= np.array([0.01, 0.02, 0.04, 0.08])
T_bar=25
coord_lim = np.array([[0,0.05], [0,0.01]], dtype = 'float')
#dt_max, x, y, t, T = laser_heat_2d(coord_lim, N_array, t_end, Nt, rho, c, k, P, w, x_start, x_end, vL, T_bar)

In [ ]:
max_temp = np.zeros((len(vL), len(P)))
for i in range (len(vL)):
  P_temp = []
  for j in range(len(P)):
    dt_max, x, y, t, T = laser_heat_2d(coord_lim, N_array, t_end, Nt, rho, c, k, P[j], w, x_start, x_end, vL[i], T_bar)
    max = np.amax(T)
    P_temp.append(max)
  max_temp[i] = P_temp
  print(max_temp)




In [ ]:
for i in range(len(vL)):
  plt.plot(P, max_temp[i], label = vL[i])
plt.legend()
plt.xlabel('Power (W)')
plt.ylabel ('Max Temperature')
plt.grid()